### 직방 서비스 원룸 데이터 수집
- 복잡한 데이터 수집 방법
- 동이름 > 매물 정보(데이터 프레임)

- 절차
    - 동이름 > 위도, 경도
    - 위도, 경도 > geohash code(지역 범위)
    - geohash > 매물 아이디
    - 매물 아이디 > 매물 정보
    
- "https://meyerweb.com/eric/tools/dencoder/" 한글 코드 변환

In [5]:
import pandas as pd
import requests

In [1]:
# 1. 동이름 > 위도, 경도

In [17]:
query = '주례3동'
url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={query}&serviceType=원룸'
response = requests.get(url)
datas = response.json()['items'][0]
lat, lng = datas['lat'], datas['lng']
lat, lng

(35.147342681884766, 129.0013885498047)

In [2]:
# 2.위도, 경도 > geohash code(지역 범위)

In [38]:
#!pip install geohash2

In [19]:
import geohash2

In [26]:
code = geohash2.encode(lat,lng, precision = 5) #precision은 영역 의미, 값이 커질수록 영역이 작아짐
code

'wy5xz'

In [3]:
# 3. geohash > 매물 아이디

In [27]:
url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={code}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
response = requests.get(url)
response

<Response [200]>

In [33]:
datas=response.json()['items']
datas[:3]

[{'lat': 35.11299030626697, 'lng': 129.01556426763474, 'item_id': 30441509},
 {'lat': 35.11346566420205, 'lng': 129.01688571315356, 'item_id': 30698305},
 {'lat': 35.11348831945383, 'lng': 129.01639097551072, 'item_id': 30560779}]

In [34]:
ids = []
for data in datas :
    ids.append(data['item_id'])
ids[:5]

[30441509, 30698305, 30560779, 30698228, 30161435]

In [37]:
# list comprehention
ids = [data["item_id"] for data in datas]
ids[:3]

# result = [data**2 for data in range(10) if data % 2]
# result

[30441509, 30698305, 30560779]

In [4]:
# 4. 매물 아이디 > 매물 정보

In [39]:
ids[:3]

[30441509, 30698305, 30560779]

In [41]:
url = 'https://apis.zigbang.com/v2/items/list'
params  = {"domain" : 'zigbang',
          "withCoalition" : 'true',
          "item_ids" : ids,}
response = requests.post(url, params)
response

<Response [200]>

In [44]:
items=response.json()['items']
item_df = pd.DataFrame(items)
item_df.head()

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,...,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new
0,None,30441509,https://ic.zigbang.com/ic/items/30441509/1.jpg,월세,월세,2000,30,22.46,"{'m2': 22.46, 'p': '6.8'}","{'m2': 22.46, 'p': '6.8'}",...,False,True,원룸,[추천],부산시 서구 서대신동3가,None,None,10,2022-02-07T16:03:04+09:00,False
1,None,30698305,https://ic.zigbang.com/ic/items/30698305/1.jpg,전세,전세,5000,0,19.24,"{'m2': 19.24, 'p': '5.8'}","{'m2': 19.24, 'p': '5.8'}",...,False,True,원룸,[추천],부산시 서구 동대신동3가,None,None,6,2022-02-18T17:03:03+09:00,False
2,None,30560779,https://ic.zigbang.com/ic/items/30560779/1.jpg,월세,월세,100,30,23.14,"{'m2': 23.14, 'p': '7'}","{'m2': 23.14, 'p': '7'}",...,False,True,원룸,[추천],부산시 서구 동대신동3가,None,None,7,2022-02-09T16:31:39+09:00,False
3,None,30698228,https://ic.zigbang.com/ic/items/30698228/1.jpg,월세,월세,200,26,12.97,"{'m2': 12.97, 'p': '3.9'}","{'m2': 12.97, 'p': '3.9'}",...,False,True,원룸,[추천],부산시 서구 동대신동3가,None,None,7,2022-02-18T17:01:01+09:00,False
4,None,30161435,https://ic.zigbang.com/ic/items/30161435/1.jpg,월세,월세,500,25,17.97,"{'m2': 17.97, 'p': '5.4'}","{'m2': 12.52, 'p': '3.8'}",...,False,True,오피스텔,[],부산시 서구 동대신동3가,None,None,5,2022-01-27T12:47:16+09:00,False


In [48]:
pd.options.display.max_columns = 30

In [62]:
columns = ['item_id', 'sales_title', 'deposit', 'rent', 'size_m2', 'floor', 
           'building_floor', 'title', 'room_type', 'address', 'is_new']
result_df = item_df[columns]
result_df = result_df[result_df['address'].str.contains('사상구')]
result_df = result_df[(result_df['deposit'] <= 500)]

result_df

,item_id,sales_title,deposit,rent,size_m2,floor,building_floor,title,room_type,address,is_new
51,30598973,월세,500,45,46.281,3,6,⭕학장동 원룸 AK빌딩H타입⭕,01,사상구 학장동,False
52,30530654,월세,300,26,19.830,3,4,⭐⭐신축 풀옵션⭐⭐보증금조절가능,01,사상구 주례동,False
53,30729714,월세,200,26,19.220,2,5,"💛가성비최고원룸💛동서,경정대 바로 밑 신축 100%🌊",01,사상구 주례동,True
54,30729767,월세,300,25,25.780,3,5,"🧡동서,경정 하나남은 호실😎신축100%",01,사상구 주례동,True
55,30715694,월세,300,22,9.920,2,5,🚩풀 옵션🚩실사🚩동서대 1분 거리 🚩,01,사상구 주례동,True
...,...,...,...,...,...,...,...,...,...,...,...
315,30507810,월세,500,44,33.900,고,9,s♥통큰♥설렘♥NEW♥유쾌♥탄탄붙박이♥화이트♥화사♥넉넉여유,01,사상구 주례동,False
316,30621341,월세,500,42,42.690,고,9,🎀보증금조절가능 🎀주례냉정1등 🎀신축1년차,01,사상구 주례동,False
317,29647955,월세,100,15,16.530,1,3,♋방크고 다락도 있는원룸♋싼방전문가♋가성비甲♋,01,사상구 주례동,True
318,30552312,월세,500,40,40.310,고,9,🎀넓은원룸 🎀보증금조절 🎀신축1년차,01,사상구 주례동,False


In [ ]:
#보증금 1억 이하, 월세 100만원 이하

In [68]:
def oneroom(address) :
    url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={address}&serviceType=원룸'
    response = requests.get(url)
    datas = response.json()['items'][0]
    lat, lng = datas['lat'], datas['lng']
    code = geohash2.encode(lat,lng, precision = 5)
    url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={code}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
    response = requests.get(url)
    datas = response.json()['items']
    ids = [data["item_id"] for data in datas]
    url = 'https://apis.zigbang.com/v2/items/list'
    params  = {"domain" : 'zigbang',
          "withCoalition" : 'true',
          "item_ids" : ids,}
    response = requests.post(url, params)
    items=response.json()['items']
    columns = ['item_id', 'sales_title', 'deposit', 'rent', 'size_m2', 'floor', 
           'building_floor', 'title', 'room_type', 'address', 'is_new']
    item_df = pd.DataFrame(items)
    result_df = item_df[columns]
    result_df = result_df[result_df['address'].str.contains('사상구')]
    return result_df

oneroom('사상구')
    

,item_id,sales_title,deposit,rent,size_m2,floor,building_floor,title,room_type,address,is_new
51,30598973,월세,500,45,46.281,3,6,⭕학장동 원룸 AK빌딩H타입⭕,01,사상구 학장동,False
52,30530654,월세,300,26,19.830,3,4,⭐⭐신축 풀옵션⭐⭐보증금조절가능,01,사상구 주례동,False
53,30729714,월세,200,26,19.220,2,5,"💛가성비최고원룸💛동서,경정대 바로 밑 신축 100%🌊",01,사상구 주례동,True
54,30729767,월세,300,25,25.780,3,5,"🧡동서,경정 하나남은 호실😎신축100%",01,사상구 주례동,True
55,30715694,월세,300,22,9.920,2,5,🚩풀 옵션🚩실사🚩동서대 1분 거리 🚩,01,사상구 주례동,True
...,...,...,...,...,...,...,...,...,...,...,...
316,30621341,월세,500,42,42.690,고,9,🎀보증금조절가능 🎀주례냉정1등 🎀신축1년차,01,사상구 주례동,False
317,29647955,월세,100,15,16.530,1,3,♋방크고 다락도 있는원룸♋싼방전문가♋가성비甲♋,01,사상구 주례동,True
318,30552312,월세,500,40,40.310,고,9,🎀넓은원룸 🎀보증금조절 🎀신축1년차,01,사상구 주례동,False
319,30687629,월세,500,35,34.620,고,9,🎀허위X 🎀보증금조절 🎀신축1년차,01,사상구 주례동,False
